In [1]:
!pip install psycopg2
!pip install getpass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [2]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import os
from getpass import getpass
import numpy as np

In [3]:
PGEND_POINT = 'f1stats.cos0zhtpa15v.us-east-1.rds.amazonaws.com' # End_point
PGDATABASE_NAME = 'F1_stats' # Database name
PGUSER_NAME = 'postgres' # UserName
PGPASSWORD = getpass('Enter database password') 

Enter database password··········


In [4]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [5]:
conn, cursor = connect()

Connected!


In [8]:
#Read in query results from each tables in the database
circuits_sql_df = pd.DataFrame(pd.read_sql_query("""SELECT * FROM circuits""",conn))
constructors_sql_df = pd.DataFrame(pd.read_sql_query("""SELECT * FROM constructors""",conn))
driver_stnd_sql_df = pd.DataFrame(pd.read_sql_query("""SELECT * FROM driver_stnd""",conn))
drivers_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM drivers""",conn))
lap_times_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM lap_times""",conn))
pit_stops_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM pit_stops""",conn))
quali_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM quali""",conn))
races_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM races""",conn))
results_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM results""",conn))
resultsplus_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM results_merged""",conn))
sprint_rslt_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM sprint_rslt""",conn))
status_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM status""",conn))
#final_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM final_merged_table""",conn))
#final2_sql_df  = pd.DataFrame(pd.read_sql_query("""SELECT * FROM big_ass_dataset_4""",conn))
final3_sql_df = pd.DataFrame(pd.read_sql_query("""SELECT * FROM outer_join_final""",conn))

idea for levels of aggregation/abstraction:
* driver x lap {500k records}
* driver x race {1100 races * 20 drivers per race = 10k}
* driver x season (or year) {50 seasons * 20 drivers = 1000}
* driver (whole career) {850 drivers total}
* drivers who have won at least 1 race {~100-130?}
* drivers who have won at least 2 race (or some other number that cuts it down to a reasonable number for display purpose [dendrogram]?)

In [9]:
final3_sql_df.columns

Index(['driverId', 'dob', 'nationality', 'dob_formatted', 'raceId', 'grid',
       'results_position', 'positionText', 'positionOrder', 'results_points',
       'results_laps', 'fastestLap', 'rank', 'race_ms',
       'results_fastestlaptime', 'fastestLapSpeed', 'statusId', 'status',
       'year', 'round', 'circuitId', 'races_name', 'date', 'dstnd_points',
       'dstnd_position', 'wins', 'lapt_lap', 'lapt_position', 'lap_ms',
       'quali_position'],
      dtype='object')

In [10]:
race_key_id = ['driverId', 'raceId']
race_level = ['round', 'circuitId','year', 'date','races_name',  'quali_position','grid', 
              'results_position', 'positionText', 'positionOrder', 'results_points', 'results_laps', 'statusId', 'status', 
              'fastestLap', 'rank', 'race_ms', 'results_fastestlaptime', 'fastestLapSpeed', 'dstnd_points', 'dstnd_position', 'wins', 'dob_formatted','nationality']

In [52]:
race_df = final3_sql_df[race_key_id + race_level].drop_duplicates()
display(race_df.shape)
race_df.head()
race_df.to_csv('race_level_aggregation.csv')

(9004, 26)

In [12]:
driver_list = race_df['driverId'].unique()
year_list = race_df['year'].unique()

In [69]:
season_cols = ['round', 'quali_position', 'grid', 'results_position', 'results_points', 'results_laps', 'rank', 'dstnd_points', 'dstnd_position', 'wins', 'dob_formatted']

In [70]:
race_data = race_df
for cols in season_cols:
  race_data[cols] = pd.to_numeric(race_df[cols], downcast='float', errors='coerce')

In [206]:
season_data = race_data[['driverId', 'year']+ season_cols]
season_data.head()

,driverId,year,round,quali_position,grid,results_position,results_points,results_laps,rank,dstnd_points,dstnd_position,wins,dob_formatted
0,1,2009.0,2.0,13.0,12.0,7.0,1.0,31.0,11.0,1.0,10.0,0.0,1985.0
1,1,2009.0,13.0,1.0,1.0,12.0,0.0,52.0,3.0,27.0,7.0,1.0,1985.0
51,1,2009.0,14.0,1.0,1.0,1.0,10.0,61.0,2.0,37.0,6.0,2.0,1985.0
112,1,2009.0,17.0,1.0,1.0,NaN,0.0,20.0,3.0,49.0,5.0,2.0,1985.0
129,1,2010.0,1.0,4.0,4.0,3.0,15.0,49.0,4.0,15.0,3.0,0.0,1985.0


In [ ]:
# max of point, laps, wins by driver by year
# unique count of results_positions, quali position by driver by year
# final value of dstnd_postion for each year (highest points equals lowest rank)

In [207]:
season_df = pd.pivot_table(season_data, values=['dstnd_points', 'wins', 'results_laps'], index=['driverId', 'year'], aggfunc={'dstnd_points':np.max, 'wins':np.max, 'results_laps':sum}) 
#'grid', 'results_position', 'results_points', 'statId', 'rank',

In [208]:
result_list=[]
grid_list = []
for num in range(1,25):
  result_list.append('res_'  + str(num))
  grid_list.append('grid_'  + str(num))

In [209]:
season_df[result_list] = 0
season_df[grid_list] = 0

In [ ]:
sd = season_data
results = sd
for y in year_list:
    rounds = season_data[season_data['year']==y]['round'].max()
    print(y, rounds)
    for d in driver_list:
      #try:
      #  season_df.loc[(d, y),'races'] = rounds
      #except:
        
      mask = (sd['driverId']==d) & (sd['year']==y)
      temp = sd[mask]
      res_dump = temp['results_position'].value_counts()
      grid_dump = temp['grid'].value_counts()
      for num in range(1,25):
        res_col = 'res_' + str(num)
        grid_col = 'grid_' + str(num)
        try:
          season_df.loc[(d, y),res_col] = res_dump.loc[num]
          season_df.loc[(d, y),grid_col] = grid_dump.loc[num]
        except:
          continue

In [230]:
season_df.head()

dstnd_points  results_laps  wins  res_1  res_2  res_3  res_4  \
driverId year                                                                   
1        2007.0         109.0        1037.0   4.0      4      5      3      1   
         2008.0          98.0        1065.0   5.0      5      2      3      0   
         2009.0          49.0         846.0   2.0      2      1      2      1   
         2010.0         240.0        1001.0   3.0      3      5      1      2   
         2011.0         227.0        1013.0   3.0      3      3      0      5   

                 res_5  res_6  res_7  ...  grid_16  grid_17  grid_18  grid_19  \
driverId year                         ...                                       
1        2007.0      1      0      1  ...        0        0        0        0   
         2008.0      3      0      1  ...        0        0        0        0   
         2009.0      0      1      1  ...        1        0        1        0   
         2010.0      2      2      0  ...        0        0        0        0   
         2011.0      2      1      1  ...        0        0        0        0   

                 grid_20  grid_21  grid_22  grid_23  grid_24  races  
driverId year                                                        
1        2007.0        0        0        0        0        0      0  
         2008.0        0        0        0        0        0      0  
         2009.0        0        0        0        0        0      0  
         2010.0        0        0        0        0        0      0  
         2011.0        0        0        0        0        0      0  

[5 rows x 52 columns]

In [231]:
season_df.to_csv('season_level_aggregation.csv')

In [263]:
career_df = pd.DataFrame(index =driver_list, columns = season_df.columns)

In [266]:

for drv in driver_list:
  try:
    career_df.loc[drv] = season_df.loc[drv].sum()
  except:
    continue

In [272]:
career_df = career_df.reset_index().rename(columns={'index':'driverId'})

In [274]:
career_df.to_csv('career_level_aggregation.csv')